In [29]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [30]:
IMGS_DIR = '../imgs'

## Coalescing Bandwidth

In [39]:
offset_gb_s = [
    83.084076,83.388924,83.416748,84.373947,84.459457,84.373947,83.333336,83.167000,83.416748,
    84.488007,83.084076,83.111702,84.345482,84.889641,84.545151,83.222366,84.203438,84.260193,
    83.892616,84.146751,83.111702,83.500336,84.545151,83.752098,83.361122,83.250084,83.472458,
    83.333336,83.277817,83.640015,83.250084,84.288605,83.780167
]

stride_gb_s = [
    84.918480,44.499821,30.048077,22.589680,18.147503,15.137753,12.980269,11.347647,11.329647,
    11.321951,11.310681,11.290759,11.252645,11.227880,11.156730,11.127431,10.869565,10.623380,
    10.252625,10.227877,9.720062,9.527439,9.126752,9.504258,8.680555,8.539126,8.241034,8.389825,
    7.911643,7.886933,7.618700,8.230995
]

offset_x = np.arange(len(offset_gb_s))
stride_x = np.arange(len(stride_gb_s))

plt.style.use('default')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5), sharey=True)

# Plot Offset Bandwidth
ax1.plot(offset_x, offset_gb_s, marker='x', linestyle='-', color='royalblue', label='Offset Access', fillstyle='none')
ax1.set_title('Offset Memory Access', fontsize=14, fontweight='bold')
ax1.set_ylabel('Bandwidth (GB/s)', fontsize=12)
ax1.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) # Hide x-axis labels
ax1.legend()
ax1.set_ylim(0, max(max(offset_gb_s), max(stride_gb_s)) * 1.1) 

# Plot Stride Bandwidth
ax2.plot(stride_x, stride_gb_s, marker='^', linestyle='--', color='crimson', label='Strided Access', fillstyle='none')
ax2.set_title('Strided Memory Access', fontsize=14, fontweight='bold')
ax2.set_ylabel('Bandwidth (GB/s)', fontsize=12)
ax2.legend()

fig.suptitle('Bandwidth Quadro T1000', fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(IMGS_DIR, 'coalescing.png'))
plt.close()

In [81]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft

In [85]:
x = np.linspace(-4, 4, 128)
vals = np.piecewise(x, [x < -1, (x >= -1) & (x <= 1), x > 1.0], [0,1,0])
ft = fft(vals)

def read_fft():
    vals = []
    with open('../data/fft_box', 'r') as f:
        for line in f:
            vals.extend([float(x) for x in line.split(' ') if len(x.strip()) != 0])
    return np.array(vals)

def plot_ffts(fname="fft_comp.png"):
    x = np.linspace(-4, 4, 128)
    vals = np.piecewise(x, [x < -1, (x >= -1) & (x <= 1), x > 1.0], [0, 1, 0])

    ft_np = np.fft.fft(vals)
    ft_own = read_fft()

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Original Signal
    axes[0].plot(x, vals, color='dodgerblue', lw=2, label='Signal')
    axes[0].grid(True, ls='--', alpha=0.7)
    axes[0].set_ylim(-0.2, 1.2)
    axes[0].legend()

    # Freqs for FFT plots
    N = len(vals)
    s_sp = x[1] - x[0]
    freqs = np.fft.fftfreq(N, d=s_sp)
    freqs_s = np.fft.fftshift(freqs)

    # NumPy FFT
    axes[1].plot(freqs_s, np.fft.fftshift(np.real(ft_np)),
                 color='mediumseagreen', lw=2, label='NumPy FFT')
    axes[1].grid(True, ls='--', alpha=0.7)
    axes[1].legend()

    # Own FFT
    axes[2].plot(freqs_s, np.fft.fftshift(np.real(ft_own)),
                 color='darkorange', lw=2, label='FFT')
    axes[2].grid(True, ls='--', alpha=0.7)
    axes[2].legend()


    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    try:
        save_path = os.path.join(os.getcwd(), fname)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    except Exception as e:
        print(f"Error saving plot: {e}")
    finally:
        plt.close(fig)

if __name__ == "__main__":
    plot_ffts("fft_comp.png")